In [18]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [20]:
load_dotenv()
KEY = os.getenv("OPENAI_API_KEY")


In [17]:
llm = ChatOpenAI(openai_api_key= KEY ,model_name='gpt-3.5-turbo',temperature=0.3)

In [44]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
        "explain":"explaination"
    },
}

In [45]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
For each question, provide:
- The question
- Four options (a, b, c, d)
- The correct answer (as a, b, c, or d)
- A brief explanation for the correct answer

Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Format your response like the RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}
"""

In [46]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [47]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [48]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [49]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE
) 

In [50]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True) 

In [51]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [52]:
file_path=r"/Users/b.anasurya/Desktop/Generative AI/MCQ_generator/data.txt"

In [53]:
file_path

'/Users/b.anasurya/Desktop/Generative AI/MCQ_generator/data.txt'

In [54]:
with open(file_path,'r') as file:
    TEXT = file.read() 

In [55]:
print(TEXT)

For the journal, see Machine Learning (journal).
"Statistical learning" redirects here. For statistical learning in linguistics, see Statistical learning in language acquisition.
Part of a series on
Machine learning
and data mining
Paradigms
Problems
Supervised learning
(classification • regression)
Clustering
Dimensionality reduction
Structured prediction
Anomaly detection
Neural networks
Reinforcement learning
Learning with humans
Model diagnostics
Mathematical foundations
Journals and conferences
Related articles
vte
Part of a series on
Artificial intelligence (AI)

Major goals
Artificial general intelligenceIntelligent agentRecursive self-improvementPlanningComputer visionGeneral game playingKnowledge representationNatural language processingRoboticsAI safety
Approaches
Applications
Philosophy
History
Glossary
vte
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gener

In [56]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer", "explain": "explaination"}}'

In [57]:
NUMBER=5 
SUBJECT="Machine Learning"
TONE="simple"

In [58]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:For the journal, see Machine Learning (journal).
"Statistical learning" redirects here. For statistical learning in linguistics, see Statistical learning in language acquisition.
Part of a series on
Machine learning
and data mining
Paradigms
Problems
Supervised learning
(classification • regression)
Clustering
Dimensionality reduction
Structured prediction
Anomaly detection
Neural networks
Reinforcement learning
Learning with humans
Model diagnostics
Mathematical foundations
Journals and conferences
Related articles
vte
Part of a series on
Artificial intelligence (AI)

Major goals
Artificial general intelligenceIntelligent agentRecursive self-improvementPlanningComputer visionGeneral game playingKnowledge representationNatural language processingRoboticsAI safety
Approaches
Applications
Philosophy
History
Glossary
vte
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms tha

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:For the journal, see Machine Learning (journal).
"Statistical learning" redirects here. For statistical learning in linguistics, see Statistical learning in language acquisition.
Part of a series on
Machine learning
and data mining
Paradigms
Problems
Supervised learning
(classification • regression)
Clustering
Dimensionality reduction
Structured prediction
Anomaly detection
Neural networks
Reinforcement learning
Learning with humans
Model diagnostics
Mathematical foundations
Journals and conferences
Related articles
vte
Part of a series on
Artificial intelligence (AI)

Major goals
Artificial general intelligenceIntelligent agentRecursive self-improvementPlanningComputer visionGeneral game playingKnowledge representationNatural language processingRoboticsAI safety
Approaches
Applications
Philosophy
History
Glossary
vte
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statist

In [59]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:5894
Prompt Tokens:4860
Completion Tokens:1034
Total Cost:0.009358000000000002


In [60]:
response

{'text': 'For the journal, see Machine Learning (journal).\n"Statistical learning" redirects here. For statistical learning in linguistics, see Statistical learning in language acquisition.\nPart of a series on\nMachine learning\nand data mining\nParadigms\nProblems\nSupervised learning\n(classification • regression)\nClustering\nDimensionality reduction\nStructured prediction\nAnomaly detection\nNeural networks\nReinforcement learning\nLearning with humans\nModel diagnostics\nMathematical foundations\nJournals and conferences\nRelated articles\nvte\nPart of a series on\nArtificial intelligence (AI)\n\nMajor goals\nArtificial general intelligenceIntelligent agentRecursive self-improvementPlanningComputer visionGeneral game playingKnowledge representationNatural language processingRoboticsAI safety\nApproaches\nApplications\nPhilosophy\nHistory\nGlossary\nvte\nMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical alg

In [61]:
quiz = json.loads(response.get("quiz"))
for q_num, q_data in quiz.items():
    print(f"Q{q_num}: {q_data['mcq']}")
    for opt, text in q_data['options'].items():
        print(f"  {opt}. {text}")
    print(f"Correct: {q_data['correct']}")
    print(f"Explanation: {q_data['explain']}\n")

Q1: What is the primary goal of machine learning?
  a. Achieving artificial intelligence
  b. Tackling solvable problems of a practical nature
  c. Focusing on symbolic approaches inherited from AI
  d. Using only neural networks for all tasks
Correct: b
Explanation: The primary goal of machine learning shifted from achieving artificial intelligence to tackling solvable problems of a practical nature.

Q2: Which field of study focuses on exploratory data analysis via unsupervised learning?
  a. Data mining
  b. Neural networks
  c. Reinforcement learning
  d. Structured prediction
Correct: a
Explanation: Data mining is a related field of study that focuses on exploratory data analysis (EDA) via unsupervised learning.

Q3: Who coined the term 'machine learning' in 1959?
  a. Alan Turing
  b. Donald Hebb
  c. Arthur Samuel
  d. Tom M. Mitchell
Correct: c
Explanation: The term 'machine learning' was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gami

In [63]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    explain = value["explain"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct,"Explanation": explain}) 

In [64]:
quiz_table_data

[{'MCQ': 'What is the primary goal of machine learning?',
  'Choices': 'a: Achieving artificial intelligence | b: Tackling solvable problems of a practical nature | c: Focusing on symbolic approaches inherited from AI | d: Using only neural networks for all tasks',
  'Correct': 'b',
  'Explanation': 'The primary goal of machine learning shifted from achieving artificial intelligence to tackling solvable problems of a practical nature.'},
 {'MCQ': 'Which field of study focuses on exploratory data analysis via unsupervised learning?',
  'Choices': 'a: Data mining | b: Neural networks | c: Reinforcement learning | d: Structured prediction',
  'Correct': 'a',
  'Explanation': 'Data mining is a related field of study that focuses on exploratory data analysis (EDA) via unsupervised learning.'},
 {'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Alan Turing | b: Donald Hebb | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c',
  'Explanation': "The term 'machin

In [67]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv", index=False)

In [70]:
print(type(quiz))

<class 'pandas.core.frame.DataFrame'>
